In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
cd /content/drive/My Drive/TFMLz/ESRGAN_oldarch

/content/drive/My Drive/TFMLz/ESRGAN_oldarch


# YUVups

In [0]:
zirr = 'V004_b'
wrkbase = '/content/'

%cd /content
!git clone https://github.com/IbrahimKurz/{zirr}.git
!chmod 755 /content/{zirr}/README.sh
!/content/{zirr}/README.sh

/content
Cloning into 'V004_b'...
remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 41 (delta 0), reused 0 (delta 0), pack-reused 35
Unpacking objects: 100% (41/41), done.
Checking out files: 100% (33/33), done.


In [0]:
deintarle = 3
episode = 4

fixcolor = 0


#===================================
DictChormaFix = {
    2:(-9,-4,-2),
    4:(7,10,-10)
                 }


zirr = 'V004_b'
wrkbase = '/content/'
import shutil
shutil.rmtree(zirr, ignore_errors=True)



import numpy as np
import cv2
from PIL import Image 
import os
import sys
import glob
import torch


Wr = 0.299
Wg = 0.587
Wb = 0.114
Umax = 0.436
Vmax = 0.615

scaleYp = 256.0/220.0
scaleUVp = 256.0/225.0


def RshapeFrame(xfr):
  bY = np.array(xfr.planes[0]).reshape((480,768))[:,:720]
  bU = np.array(xfr.planes[1]).reshape((480,384))[:240,:360]
  bV = np.array(xfr.planes[2]).reshape((480,384))[:240,:360]
  return bY,bU,bV

def interlace(imgL, imgR, h, w):
    inter = np.empty((h, w), imgL.dtype)
    inter[:h:2, :w] = imgR[:h:2, :w]
    inter[1:h:2, :w] = imgL[1:h:2, :w]
    return inter

def testpics(starti,typ=0):
  vstp = 8
  vdir = 'wY'

  if typ == 1:
    vstp = 10
    vdir = 'oY'


  showp = vstp*   starti
  flist = os.listdir(wrkbase+vdir)
  flist.sort()

  for i in range(showp,showp+vstp):
    print(flist[i])
    display(Image.open(wrkbase+vdir+'/'+flist[i]))


def toRGB444(fiY,fiU=None,fiV,gshow,slif=None):

  usefile = isinstance(fiY, str)
  if fiU is None:
    if usefile:
    else:
  else:
    if usefile:
      tYpl = cv2.imread(fiY, cv2.IMREAD_GRAYSCALE).astype(np.float32)
      tUpl = cv2.imread(fiU, cv2.IMREAD_GRAYSCALE).astype(np.float32)
      tVpl = cv2.imread(fiV, cv2.IMREAD_GRAYSCALE).astype(np.float32)
      tUpl -= 128.0
      tVpl -= 128.0
    else:
      tYpl = fiY
      tUpl = fiU
      tVpl = fiV


  

  if slif is not None:
    tYpl+=slif[0]
    tUpl+=slif[1]
    tVpl+=slif[2]


  tfR = tYpl + tVpl*(1-Wr)/Vmax
  tfG = tYpl - (tUpl*Wb*(1-Wb)/Umax+tVpl*Wr*(1-Wr)/Vmax)/Wg
  tfB = tYpl + tUpl*(1-Wb)/Umax

  domp = np.transpose(np.asarray([tfR,tfG,tfB]), (1, 2, 0)).clip(0,255.0)

  if gshow==1:
    print(fiY)
    display(Image.fromarray(domp.astype(np.uint8)))
  else:
    cv2.imwrite(wrkbase+'ottest.png', domp[:,:,[2,1,0]])

  return domp
  


def toRGB422_f(fiY,fiU,fiV,gshow,slif=None):
  Ypl = cv2.imread(fiY, cv2.IMREAD_GRAYSCALE).astype(np.float32)
  Upl = cv2.imread(fiU, cv2.IMREAD_GRAYSCALE).astype(np.float32)
  Vpl = cv2.imread(fiV, cv2.IMREAD_GRAYSCALE).astype(np.float32)

  if scaleYUV == 1:
    Ypl-=16
    Ypl*=scaleYp
    Upl-=16
    Upl*=scaleUVp
    Vpl-=16
    Vpl*=scaleUVp

  if slif is not None:
    tYpl+=slif[0]
    tUpl+=slif[1]
    tVpl+=slif[2]


  Upl -= 128.0
  Vpl -= 128.0

  Fsiz = toRGB422(Ypl,Upl,Vpl)

  domp = np.transpose(Fsiz, (1, 2, 0)).clip(0,255.0)

  if gshow==1:
    print(fiY)
    display(Image.fromarray(domp.astype(np.uint8)))
  else:
    cv2.imwrite(wrkbase+'ottest422.png', domp[:,:,[2,1,0]])

  return domp


def toRGB422(tYpl,tUpl,tVpl):
  tUpl=np.repeat(tUpl,2,0)
  tUpl=np.repeat(tUpl,2,1)
  tVpl=np.repeat(tVpl,2,0)
  tVpl=np.repeat(tVpl,2,1)



  tfR = tYpl + tVpl*(1-Wr)/Vmax
  tfG = tYpl - (tUpl*Wb*(1-Wb)/Umax+tVpl*Wr*(1-Wr)/Vmax)/Wg
  tfB = tYpl + tUpl*(1-Wb)/Umax

  #cv2.imwrite(wrkbase+'ottest422.png', np.transpose(np.asarray([tfB,tfG,tfR]), (1, 2, 0)).clip(0,255.0))

  return np.asarray([tfR,tfG,tfB])
 
def dumpYUV(output,patha):
  fY = Wr*output[0]+Wg*output[1]+Wb*output[2]
  fU = Umax*(output[2]-fY)/(1-Wb)
  fV = Vmax*(output[0]-fY)/(1-Wr)
  fU+=0.5
  fV+=0.5

  cv2.imwrite('/content/2xM/Y/'+patha, fY.clip(0,1)*255.0)
  cv2.imwrite('/content/2xM/U/'+patha, fU.clip(0,1)*255.0)
  cv2.imwrite('/content/2xM/V/'+patha, fV.clip(0,1)*255.0)



scaleYUV = 1

colofix = DictChormaFix[episode]

# deinterlace_ffms2

In [0]:
!sudo apt-get -y install libffms2-4 
!pip install ffms2
!mkdir wY
!mkdir wU
!mkdir wV

import ffms2


vsource = ffms2.VideoSource(wrkbase+'wrk.m2v')
fcot = vsource.properties.NumFrames




idx=0
while idx < fcot:  
  bY, bU, bV = RshapeFrame(vsource.get_frame(idx))
  if idx%5 == deintarle:
    idx+=1
    nxbY, nxbU, nxbV = RshapeFrame(vsource.get_frame(idx))
    bY = interlace(bY,nxbY,480,720)
    bU = interlace(bU,nxbU,240,360)
    bV = interlace(bV,nxbV,240,360)
  
  

  strfmt = '%sw%s/x%05d.png' % (wrkbase,'%s',idx)
  cv2.imwrite(strfmt % 'Y',bY)
  cv2.imwrite(strfmt % 'U',bU)
  cv2.imwrite(strfmt % 'V',bV)
  idx+=1



!mkdir {wrkbase}2x
!mkdir {wrkbase}2x/Y
!mkdir {wrkbase}2x/U
!mkdir {wrkbase}2x/V

# deinterlace_ffmpeg

In [0]:
!mkdir oY
!mkdir oU
!mkdir oV
!mkdir wY
!mkdir wU
!mkdir wV
!ffmpeg -vsync 0 -i wrk.m2v -filter_complex "extractplanes=y+u+v[y][u][v]" -map "[y]" 'oY/x%05d.png' -map "[u]" 'oU/x%05d.png' -map "[v]" 'oV/x%05d.png'

In [0]:






#######################

flist = os.listdir(wrkbase+'oY')
flist.sort()

maefrmY = []
maefrmU = []
maefrmV = []

idx = 0
for fi in flist:
  gyi = idx%5
  if gyi==deintarle:
    maefrmY = cv2.imread(wrkbase+'oY/'+fi, cv2.IMREAD_GRAYSCALE).astype(np.uint8)
    maefrmU = cv2.imread(wrkbase+'oU/'+fi, cv2.IMREAD_GRAYSCALE).astype(np.uint8)
    maefrmV = cv2.imread(wrkbase+'oV/'+fi, cv2.IMREAD_GRAYSCALE).astype(np.uint8)
  elif gyi==deintarle+1:
    disfrmY = cv2.imread(wrkbase+'oY/'+fi, cv2.IMREAD_GRAYSCALE).astype(np.uint8)
    disfrmU = cv2.imread(wrkbase+'oU/'+fi, cv2.IMREAD_GRAYSCALE).astype(np.uint8)
    disfrmV = cv2.imread(wrkbase+'oV/'+fi, cv2.IMREAD_GRAYSCALE).astype(np.uint8)

    cv2.imwrite(wrkbase+'wY/'+fi,interlace(maefrmY,disfrmY,480,720))
    cv2.imwrite(wrkbase+'wU/'+fi,interlace(maefrmU,disfrmU,240,360))
    cv2.imwrite(wrkbase+'wV/'+fi,interlace(maefrmV,disfrmV,240,360))
  else:
    os.link(wrkbase+'oY/'+fi,wrkbase+'wY/'+fi)
    os.link(wrkbase+'oU/'+fi,wrkbase+'wU/'+fi)
    os.link(wrkbase+'oV/'+fi,wrkbase+'wV/'+fi)

  idx+=1



In [0]:
flist_ms = os.listdir(wrkbase+'wY')
flist_ms.sort()


#shutil.copy('/content/wYa/x00200.png','/content/wYa/x00100.png')
'''
flist_pg = os.listdir(wrkbase+'wYa')
flist_pg.sort()

print(len(flist_ms))
print(len(flist_pg))

gito = 1538

strms = wrkbase+'wY/'+flist_ms[gito+1]
strpg = wrkbase+'wYa/'+flist_pg[gito]

print(strms)
display(Image.open(strms))
print(strpg)
display(Image.open(strpg))
'''

flist_ms = flist_ms[10:]

for fi in flist_ms:
  img_ms = cv2.imread(wrkbase+'wY/'+fi, cv2.IMREAD_GRAYSCALE).astype(np.uint8)
  img_pg = cv2.imread(wrkbase+'wYa/'+fi, cv2.IMREAD_GRAYSCALE).astype(np.uint8)
  if not (img_ms==img_pg).all():
    print('not eq '+fi)

In [0]:
shutil.rmtree('oU', ignore_errors=True)
shutil.rmtree('oY', ignore_errors=True)
shutil.rmtree('oV', ignore_errors=True)





In [0]:
'''
shutil.rmtree(wrkbase+'wU', ignore_errors=True)
shutil.rmtree(wrkbase+'wY', ignore_errors=True)
shutil.rmtree(wrkbase+'wV', ignore_errors=True)
!mkdir {wrkbase}wY
!mkdir {wrkbase}wU
!mkdir {wrkbase}wV
'''

# SRproc

In [0]:
testpics(1475)

In [0]:
cd /content/drive/My Drive/TFMLz/ESRGAN_oldarch

/content/drive/My Drive/TFMLz/ESRGAN_oldarch


In [0]:
'''
import numpy as np
import cv2
from PIL import Image 
import os
import sys
import glob
import torch
'''

from google.colab import files
import architecture as arch



def mkESRGAN(model_path,scale):

  if not os.path.isfile(model_path):
    model_path = 'models/'+model_path+'.pth'

  
  model = arch.RRDB_Net(3, 3, 64, 23, gc=32, upscale=scale, norm_type=None, act_type='leakyrelu', \
                        mode='CNA', res_scale=1, upsample_mode='upconv')
  model.load_state_dict(torch.load(model_path), strict=True)
  model.eval()
  for k, v in model.named_parameters():
      v.requires_grad = False
  return model







flist = os.listdir(wrkbase+'wY')
flist.sort()

device = torch.device('cuda')  # if you want to run on CPU, change 'cuda' -> cpu
####################

m4x =  mkESRGAN('DigiPaint35000',4).to(device) #  2x_Faithful_v2_130000_G' '/content/falcoon300.pth'

m2x =  mkESRGAN('LADDIER1_282500_G',4).to(device)    #'LADDIER1_282500_G' #'/content/4x_DigitalFrames_2.1.pth'

###################

#test
flist = flist[1475*8:1475*8+100]#flist[1392*8:1392*8+2000]
#test


for fi in flist:
  Ypl = cv2.imread(wrkbase+'wY/'+fi, cv2.IMREAD_GRAYSCALE).astype(np.float32)
  Upl = cv2.imread(wrkbase+'wU/'+fi, cv2.IMREAD_GRAYSCALE).astype(np.float32)
  Vpl = cv2.imread(wrkbase+'wV/'+fi, cv2.IMREAD_GRAYSCALE).astype(np.float32)

  if scaleYUV == 1:
    Ypl-=16
    Ypl*=scaleYp
    Upl-=16
    Upl*=scaleUVp
    Vpl-=16
    Vpl*=scaleUVp


  Upl -= 128.0
  Vpl -= 128.0


  #======mdl2x=========
  Fsiz = toRGB422(Ypl,Upl,Vpl)
  img = torch.from_numpy(Fsiz/255.0)
  img_LR = img.unsqueeze(0)
  img_LR = img_LR.to(device)

  output = m2x(img_LR).data.squeeze().cpu().numpy() #.squeeze().float().cpu().numpy()
 
  fY = (Wr*output[0]+Wg*output[1]+Wb*output[2])*255.0

  #======mdl2xEND=========

  Yplh =  cv2.resize(Ypl, (360, 240), interpolation=cv2.INTER_LINEAR)



  #Ypatch = (cv2.resize(Ypl, (1440, 960), interpolation=cv2.INTER_LINEAR)-cv2.resize(Yplh, (1440, 960), interpolation=cv2.INTER_LINEAR))/255.0

  Ypatch = (cv2.resize(fY, (1440, 960), interpolation=cv2.INTER_LINEAR)-cv2.resize(Yplh, (1440, 960), interpolation=cv2.INTER_LINEAR))/255.0


  fR = Yplh + Vpl*(1-Wr)/Vmax
  fG = Yplh - (Upl*Wb*(1-Wb)/Umax+Vpl*Wr*(1-Wr)/Vmax)/Wg
  fB = Yplh + Upl*(1-Wb)/Umax

  timp = np.asarray([fR,fG,fB])

  img = torch.from_numpy(timp/255.0)
  img_LR = img.unsqueeze(0)
  img_LR = img_LR.to(device)

  output = m4x(img_LR).data.squeeze().cpu().numpy() #.squeeze().float().cpu().numpy()
 
  fY = Wr*output[0]+Wg*output[1]+Wb*output[2]

  '''
  fU = -0.147*output[0]-0.289*output[1]+0.436*output[2]
  fV = 0.615*output[0]-0.515*output[1]-0.100*output[2]
  '''

  fU = Umax*(output[2]-fY)/(1-Wb)
  fV = Vmax*(output[0]-fY)/(1-Wr)

  '''
  print('Y: '+str(np.max(fY))+', '+str(np.min(fY))+
        '\nU: '+str(np.max(fU))+', '+str(np.min(fU))+
        '\nV: '+str(np.max(fV))+', '+str(np.min(fV)))
  '''

  fU+=0.5
  fV+=0.5
  fY+=Ypatch

  '''
  np.savez_compressed(wrkbase+'2x/Y/'+fi+'.npz', (fY.clip(0,1)*255.0).astype(np.uint8))
  np.savez_compressed(wrkbase+'2x/U/'+fi+'.npz', (fU.clip(0,1)*255.0).astype(np.uint8))
  np.savez_compressed(wrkbase+'2x/V/'+fi+'.npz', (fV.clip(0,1)*255.0).astype(np.uint8))
  '''

 
  fY*=255.0
  fU*=255.0
  fV*=255.0

  if fixcolor == 1:
    fY+=colofix[0]
    fU+=colofix[1]
    fV+=colofix[2]


  fY = cv2.resize(fY, (1080, 720), interpolation=cv2.INTER_LINEAR)
  fU = cv2.resize(fU, (1080, 720), interpolation=cv2.INTER_LINEAR)
  fV = cv2.resize(fV, (1080, 720), interpolation=cv2.INTER_LINEAR)



  cv2.imwrite(wrkbase+'2x/Y/'+fi, fY.clip(0,255))
  cv2.imwrite(wrkbase+'2x/U/'+fi, fU.clip(0,255))
  cv2.imwrite(wrkbase+'2x/V/'+fi, fV.clip(0,255))
  #toRGB444(wrkbase+'2x/Y/'+fi,wrkbase+'2x/U/'+fi,wrkbase+'2x/V/'+fi,gshow=1)



  
  

  #break

In [0]:
fxxi = 'x14800.png'
_=toRGB422_f(wrkbase+'wY/'+fxxi,wrkbase+'wU/'+fxxi,wrkbase+'wV/'+fxxi,gshow=1)
_=toRGB444(wrkbase+'2x/Y/'+fxxi,wrkbase+'2x/U/'+fxxi,wrkbase+'2x/V/'+fxxi,gshow=1,slif=(0,0,5))

In [0]:
!7z e '/content/drive/My Drive/TFMLz/ESRGAN_oldarch/models/models.7z' -o/content
#falcoon300.7z
#detoon.rar

In [0]:
#print(os.listdir(wrkbase+'4x'))
#from google.colab import files
#files.download(wrkbase+'wU/x00001.png')

In [0]:
!mkdir {wrkbase}oY
!cp {wrkbase}oYr/0155*.pgm {wrkbase}oY

# OLDarch

In [0]:
#!pip3 install numpy opencv-python
#!pip3 install torch torchvision
mkdir /content/sample_data/img

In [0]:
cd /content/drive/My Drive/TFMLz/ESRGAN_oldarch

/content/drive/My Drive/TFMLz/ESRGAN_oldarch


In [0]:
import sys
import os.path
import glob
import cv2
import numpy as np
import torch
import architecture as arch


device = torch.device('cuda')  # if you want to run on CPU, change 'cuda' -> cpu

test_img_folder = '/content/sample_data/img/*'

#!wget -P /content/sample_data/img/ http://muryouav.avximg.com/2018-05/upload/86fb6f00e0550fbf1d784d485ac7df3c516e2140.jpg


model_path = '4x_Spongebob_v6_190000_G' #2x_Faithful_v2_130000_G'

model = arch.RRDB_Net(3, 3, 64, 23, gc=32, upscale=4, norm_type=None, act_type='leakyrelu', \
                        mode='CNA', res_scale=1, upsample_mode='upconv')
model.load_state_dict(torch.load('models/'+model_path+'.pth'), strict=True)
model.eval()
for k, v in model.named_parameters():
    v.requires_grad = False
model = model.to(device)

#print('Model path {:s}. \nTesting...'.format(model_path))

idx = 0
for path in glob.glob(test_img_folder):
    idx += 1
    base = os.path.splitext(os.path.basename(path))[0]
    print(idx, base)
    # read image
    img = cv2.imread(path, cv2.IMREAD_COLOR)
    img = img * 1.0 / 255
    img = torch.from_numpy(np.transpose(img[:, :, [2, 1, 0]], (2, 0, 1))).float()
    img_LR = img.unsqueeze(0)
    img_LR = img_LR.to(device)

    output = model(img_LR).data.squeeze().float().cpu().clamp_(0, 1).numpy()
    output = np.transpose(output[[2, 1, 0], :, :], (1, 2, 0))
    output = (output * 255.0).round()
    cv2.imwrite('results/{:s}'.format(base)+'_'+model_path+'.png', output)

1 mmg


# CurArch

In [0]:
cd /content/drive/My Drive/TFMLz/ESRGAN

/content/drive/My Drive/TFMLz/ESRGAN


In [0]:
import os.path as osp
import glob
import cv2
import numpy as np
import torch
import RRDBNet_arch as arch
import sys

model_path = 'RRDB_ESRGAN_x4'
device = torch.device('cuda')  # if you want to run on CPU, change 'cuda' -> cpu
# device = torch.device('cpu')

!wget -P /content/sample_data/img/ http://img.eromenskan.com/wp-content/uploads/gazo/575/073_20140118185830ab5.jpg

test_img_folder = '/content/sample_data/img/*'

model = arch.RRDBNet(3, 3, 64, 23, gc=32)
model.load_state_dict(torch.load('models/'+model_path+'.pth'), strict=True)
model.eval()
model = model.to(device)

print('Model path {:s}. \nTesting...'.format(model_path))

idx = 0
for path in glob.glob(test_img_folder):
    idx += 1
    base = osp.splitext(osp.basename(path))[0]
    print(idx, base)
    # read images
    img = cv2.imread(path, cv2.IMREAD_COLOR)
    img = img * 1.0 / 255
    img = torch.from_numpy(np.transpose(img[:, :, [2, 1, 0]], (2, 0, 1))).float()
    img_LR = img.unsqueeze(0)
    img_LR = img_LR.to(device)

    with torch.no_grad():
        output = model(img_LR).data.squeeze().float().cpu().clamp_(0, 1).numpy()
    output = np.transpose(output[[2, 1, 0], :, :], (1, 2, 0))
    output = (output * 255.0).round()
    cv2.imwrite('results/{:s}'.format(base)+'_'+model_path+'.png', output)

# SloMo2019

In [0]:
import modelz
import datas
import configs

from PIL import Image
import torch
import torchvision
import torchvision.transforms as TF
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import time
import os
from math import log10
import numpy as np
import datetime
from utils.config import Config
import sys

import time

import cv2

# loading configures


# args = parser.parse_config()

config = Config.from_file('configs/config_test.py')

# preparing datasets
normalize1 = TF.Normalize(config.mean, [1.0, 1.0, 1.0])
normalize2 = TF.Normalize([0, 0, 0], config.std)
trans = TF.Compose([TF.ToTensor(), normalize1, normalize2, ])

revmean = [-x for x in config.mean]
revstd = [1.0 / x for x in config.std]
revnormalize1 = TF.Normalize([0.0, 0.0, 0.0], revstd)
revnormalize2 = TF.Normalize(revmean, [1.0, 1.0, 1.0])
revNormalize = TF.Compose([revnormalize1, revnormalize2])

revtrans = TF.Compose([revnormalize1, revnormalize2]) #TF.ToPILImage()

testset = datas.MySequence(config.testset_root, trans, config.test_size, config.test_crop_size, config.inter_frames)
sampler = torch.utils.data.SequentialSampler(testset)
validationloader = torch.utils.data.DataLoader(testset, sampler=sampler, batch_size=1, shuffle=False, num_workers=1)


# model
model = getattr(modelz, config.model)(config.pwc_path).cuda()
model = nn.DataParallel(model)

tot_time = 0
tot_frames = 0

print('Everything prepared. Ready to test...')



!mkdir /content/2xM
!mkdir /content/2xM/Y
!mkdir /content/2xM/U
!mkdir /content/2xM/V

rg1 = range(1)
rg2 = range(2)

def generate():
    global tot_time, tot_frames
    retImg = []
   
    

    vrgg = rg1
    vrgk = 1
    

    with torch.no_grad():
        for validationIndex, validationData in enumerate(validationloader, 0):
            sample, psyh = validationData

            naubase = psyh[0][0]
            frame0 = sample[0]
            frame1 = sample[1]
            frame2 = sample[2]
            frame3 = sample[3]

            #print(str(frame0.dtype)+',lay='+str(frame0.layout))
            #print(frame0)

            I0 = frame0.cuda()
            I3 = frame3.cuda()

            I1 = frame1.cuda()
            I2 = frame2.cuda()

            
            print(naubase)
            revtrans(I0.clone().cpu()[0]) #.save(svba)
            #dumpYUV(vymg,naubase)
            os.link('/content/2x/Y/'+naubase,'/content/2xM/Y/'+naubase)
            os.link('/content/2x/U/'+naubase,'/content/2xM/U/'+naubase)
            os.link('/content/2x/V/'+naubase,'/content/2xM/V/'+naubase)

            
                
            

            
            for tt in vrgg:
                x = config.inter_frames
                t = 1.0/(x+1) * (tt + 1)
                #print(t)


                # record duration time
                start_time = time.time()

                output = model(I0, I1, I2, I3, t)
                It_warp = output
                
                tot_time += (time.time() - start_time)
                tot_frames += 1
                

                
                vymg2 = revtrans(It_warp.cpu()[0]).numpy()
                dumpYUV(vymg2,psyh[1][0]+ '_' + str(tt) + '.png')




            
            if vrgk==1:
              vrgk = 2
              vrgg = rg2
            elif vrgk == 2:
              vrgk = 1
              vrgg = rg1
               
               
                    
def test():

    dict1 = torch.load(config.checkpoint)
    model.load_state_dict(dict1['model_state_dict'])

   
    generate()


test()

print ('Avg time is {} second'.format(tot_time/tot_frames))


In [0]:
flist = os.listdir('/content/2xM/Y')
flist.sort()



idx = 1
for fi in flist:
  vopng = '%05d.png' % idx
  os.rename('/content/2xM/Y/'+fi,'/content/2xM/Y/'+vopng)
  os.rename('/content/2xM/U/'+fi,'/content/2xM/U/'+vopng)
  os.rename('/content/2xM/V/'+fi,'/content/2xM/V/'+vopng)
  idx+=1

# To2160

In [0]:
flist = os.listdir(wrkbase+'2xM/Y')
flist.sort()



device = torch.device('cuda')  # if you want to run on CPU, change 'cuda' -> cpu
####################



m2x =  mkESRGAN('/content/4x_DigitalFrames_2.1.pth',4).to(device)    #'LADDIER1_282500_G'

for fi in flist:
  Ypl = cv2.imread(wrkbase+'2xM/Y/'+fi, cv2.IMREAD_GRAYSCALE).astype(np.float32)
  Upl = cv2.imread(wrkbase+'2xM/U/'+fi, cv2.IMREAD_GRAYSCALE).astype(np.float32)
  Vpl = cv2.imread(wrkbase+'2xM/V/'+fi, cv2.IMREAD_GRAYSCALE).astype(np.float32)

  Upl -= 128.0
  Vpl -= 128.0

  fR = Ypl + Vpl*(1-Wr)/Vmax
  fG = Ypl - (Upl*Wb*(1-Wb)/Umax+Vpl*Wr*(1-Wr)/Vmax)/Wg
  fB = Ypl + Upl*(1-Wb)/Umax

  timp = np.asarray([fR,fG,fB])

  img = torch.from_numpy(timp/255.0)
  img_LR = img.unsqueeze(0)
  img_LR = img_LR.to(device)

  output = m2x(img_LR).data.squeeze().cpu().numpy()
 
  fY = Wr*output[0]+Wg*output[1]+Wb*output[2]
  fU = Umax*(output[2]-fY)/(1-Wb)
  fV = Vmax*(output[0]-fY)/(1-Wr)

  fU+=0.5
  fV+=0.5

  fY*=255.0
  fU*=255.0
  fV*=255.0

  fY = cv2.resize(fY, (2960, 2160), interpolation=cv2.INTER_LINEAR)
  fU = cv2.resize(fU, (1480, 1080), interpolation=cv2.INTER_LINEAR)
  fV = cv2.resize(fV, (1480, 1080), interpolation=cv2.INTER_LINEAR)

  #fY/=scaleYp
  #fY+=16

  fU/=scaleUVp
  fV/=scaleUVp

  #fU+=16
  #fV+=16

  cv2.imwrite(wrkbase+'2xM/Y/'+fi, fY.clip(0,255))
  cv2.imwrite(wrkbase+'2xM/U/'+fi, fU.clip(0,255))
  cv2.imwrite(wrkbase+'2xM/V/'+fi, fV.clip(0,255))
  


In [0]:
!ffmpeg -framerate 60 -f image2 -i /content/2xM/Y/%05d.png -framerate 60 -f image2 -i /content/2xM/U/%05d.png -framerate 60 -f image2 -i /content/2xM/V/%05d.png -filter_complex "[0][1][2]mergeplanes=0x001020:yuv420p,pad=3840:2160:0:0:black" -c:v libx264 -crf 5 -shortest izo.mkv

ffmpeg version 3.4.6-0ubuntu0.18.04.1 Copyright (c) 2000-2019 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.3.0-16ubuntu3)
  configuration: --prefix=/usr --extra-version=0ubuntu0.18.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --ena

In [0]:
from google.colab import files
files.download('/content/2xM/Y/00025.png')
#import shutil
#shutil.rmtree('/content/2xM', ignore_errors=True)

In [0]:
antopo = os.listdir('/content/2xM/Y')
antopo.sort()

In [0]:
nstai = 18
nitopo = antopo[nstai:nstai+4]

for fxxi in nitopo:
  _=toRGB444('/content/2xM/Y/'+fxxi,'/content/2xM/U/'+fxxi,'/content/2xM/V/'+fxxi,gshow=1)#,slif=DictChormaFix[4])

In [0]:
shutil.rmtree('/content/2xM', ignore_errors=True)